In [1]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5476,2020-11-10T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1348,195,1543,21289,...,893,1201,9928,735,20749.0,12746.0,33495,770408,535921.0,NaN
5477,2020-11-10T17:00:00,ITA,9,Toscana,43.769231,11.255889,1582,242,1824,44490,...,1834,2223,19451,1648,54431.0,12982.0,67413,1241973,813816.0,effettuati 33301 tamponi rapidi antigenici di...
5478,2020-11-10T17:00:00,ITA,10,Umbria,43.106758,12.388247,363,66,429,9711,...,144,485,5392,206,4484.0,11254.0,15738,336098,192401.0,NaN
5479,2020-11-10T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,161,15,176,2123,...,33,127,1946,218,3936.0,527.0,4463,46630,27907.0,NaN


In [4]:
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5476,2020-11-10T17:00:00,19,Sicilia,1348,195,1543,21289,22832,893,1201,9928,735,20749.0,12746.0,33495,770408,535921.0
5477,2020-11-10T17:00:00,9,Toscana,1582,242,1824,44490,46314,1834,2223,19451,1648,54431.0,12982.0,67413,1241973,813816.0
5478,2020-11-10T17:00:00,10,Umbria,363,66,429,9711,10140,144,485,5392,206,4484.0,11254.0,15738,336098,192401.0
5479,2020-11-10T17:00:00,2,Valle d'Aosta,161,15,176,2123,2299,33,127,1946,218,3936.0,527.0,4463,46630,27907.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-10', '2020-11-09')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-10T17:00:00,1,Piemonte,4715,325,5040,53795,58835,2579,3659,40737,4742,44444.0,59870.0,104314,1178337,745706.0
1,2020-11-10T17:00:00,2,Valle d'Aosta,161,15,176,2123,2299,33,127,1946,218,3936.0,527.0,4463,46630,27907.0
2,2020-11-10T17:00:00,3,Lombardia,6682,708,7390,128115,135505,4046,10955,122410,18571,213059.0,63427.0,276486,3324961,2039672.0
3,2020-11-10T17:00:00,5,Veneto,1596,204,1800,48450,50250,-2912,2763,31362,2643,26311.0,57944.0,84255,2479307,962389.0
4,2020-11-10T17:00:00,6,Friuli Venezia Giulia,336,47,383,7458,7841,9,482,7213,476,13482.0,2048.0,15530,589577,245027.0
5,2020-11-10T17:00:00,7,Liguria,1315,92,1407,11929,13336,615,1172,22828,1976,26677.0,11463.0,38140,496323,253455.0
6,2020-11-10T17:00:00,8,Emilia-Romagna,2020,215,2235,39545,41780,2150,2430,29230,4845,51310.0,24545.0,75855,1763487,931259.0
7,2020-11-10T17:00:00,9,Toscana,1582,242,1824,44490,46314,1834,2223,19451,1648,54431.0,12982.0,67413,1241973,813816.0
8,2020-11-10T17:00:00,10,Umbria,363,66,429,9711,10140,144,485,5392,206,4484.0,11254.0,15738,336098,192401.0
9,2020-11-10T17:00:00,11,Marche,525,75,600,10289,10889,438,504,7599,1067,19405.0,150.0,19555,349370,205090.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-09T17:00:00,1,Piemonte,4540,312,4852,51404,56256,2059,2876,39721,4678,43124.0,57531.0,100655,1162525,735109.0
1,2020-11-09T17:00:00,2,Valle d'Aosta,159,15,174,2092,2266,94,201,1863,207,3822.0,514.0,4336,45993,27671.0
2,2020-11-09T17:00:00,3,Lombardia,6414,670,7084,124375,131459,-951,4777,115630,18442,203758.0,61773.0,265531,3277767,2014134.0
3,2020-11-09T17:00:00,5,Veneto,1464,195,1659,51503,53162,2192,2223,25735,2595,26203.0,55289.0,81492,2462529,953936.0
4,2020-11-09T17:00:00,6,Friuli Venezia Giulia,319,46,365,7467,7832,191,320,6760,456,13080.0,1968.0,15048,583139,243252.0
5,2020-11-09T17:00:00,7,Liguria,1387,92,1479,11242,12721,313,538,22310,1937,25834.0,11134.0,36968,489301,249942.0
6,2020-11-09T17:00:00,8,Emilia-Romagna,1939,204,2143,37487,39630,1778,2025,28978,4824,50134.0,23298.0,73432,1740948,922303.0
7,2020-11-09T17:00:00,9,Toscana,1542,234,1776,42704,44480,1033,2244,19116,1594,52685.0,12505.0,65190,1227725,806480.0
8,2020-11-09T17:00:00,10,Umbria,360,64,424,9572,9996,196,314,5060,197,4303.0,10950.0,15253,330852,189991.0
9,2020-11-09T17:00:00,11,Marche,519,71,590,9861,10451,210,262,7543,1057,18940.0,111.0,19051,346762,203665.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-10T17:00:00,1,Piemonte,4715,325,5040,53795,58835,2579,3659,40737,4742,44444.0,59870.0,104314,1178337,745706.0
1,2020-11-10T17:00:00,2,Valle d'Aosta,161,15,176,2123,2299,33,127,1946,218,3936.0,527.0,4463,46630,27907.0
2,2020-11-10T17:00:00,3,Lombardia,6682,708,7390,128115,135505,4046,10955,122410,18571,213059.0,63427.0,276486,3324961,2039672.0
3,2020-11-10T17:00:00,4,Trentino-Alto Adige,717,65,782,10690,11472,84,618,13171,827,20337.0,5133.0,25470,572731,NaN
4,2020-11-10T17:00:00,5,Veneto,1596,204,1800,48450,50250,-2912,2763,31362,2643,26311.0,57944.0,84255,2479307,962389.0
5,2020-11-10T17:00:00,6,Friuli Venezia Giulia,336,47,383,7458,7841,9,482,7213,476,13482.0,2048.0,15530,589577,245027.0
6,2020-11-10T17:00:00,7,Liguria,1315,92,1407,11929,13336,615,1172,22828,1976,26677.0,11463.0,38140,496323,253455.0
7,2020-11-10T17:00:00,8,Emilia-Romagna,2020,215,2235,39545,41780,2150,2430,29230,4845,51310.0,24545.0,75855,1763487,931259.0
8,2020-11-10T17:00:00,9,Toscana,1582,242,1824,44490,46314,1834,2223,19451,1648,54431.0,12982.0,67413,1241973,813816.0
9,2020-11-10T17:00:00,10,Umbria,363,66,429,9711,10140,144,485,5392,206,4484.0,11254.0,15738,336098,192401.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-09T17:00:00,1,Piemonte,4540,312,4852,51404,56256,2059,2876,39721,4678,43124.0,57531.0,100655,1162525,735109.0
1,1,2020-11-09T17:00:00,2,Valle d'Aosta,159,15,174,2092,2266,94,201,1863,207,3822.0,514.0,4336,45993,27671.0
2,2,2020-11-09T17:00:00,3,Lombardia,6414,670,7084,124375,131459,-951,4777,115630,18442,203758.0,61773.0,265531,3277767,2014134.0
3,19,2020-11-09T17:00:00,4,Trentino-Alto Adige,686,65,751,10637,11388,648,840,12648,816,19836.0,5016.0,24852,568496,NaN
4,3,2020-11-09T17:00:00,5,Veneto,1464,195,1659,51503,53162,2192,2223,25735,2595,26203.0,55289.0,81492,2462529,953936.0
5,4,2020-11-09T17:00:00,6,Friuli Venezia Giulia,319,46,365,7467,7832,191,320,6760,456,13080.0,1968.0,15048,583139,243252.0
6,5,2020-11-09T17:00:00,7,Liguria,1387,92,1479,11242,12721,313,538,22310,1937,25834.0,11134.0,36968,489301,249942.0
7,6,2020-11-09T17:00:00,8,Emilia-Romagna,1939,204,2143,37487,39630,1778,2025,28978,4824,50134.0,23298.0,73432,1740948,922303.0
8,7,2020-11-09T17:00:00,9,Toscana,1542,234,1776,42704,44480,1033,2244,19116,1594,52685.0,12505.0,65190,1227725,806480.0
9,8,2020-11-09T17:00:00,10,Umbria,360,64,424,9572,9996,196,314,5060,197,4303.0,10950.0,15253,330852,189991.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-10T17:00:00,1,Piemonte,4715,325,5040,53795,58835,2579,3659,40737,4742,44444.0,59870.0,104314,1178337,745706.0,4341375
1,2020-11-10T17:00:00,2,Valle d'Aosta,161,15,176,2123,2299,33,127,1946,218,3936.0,527.0,4463,46630,27907.0,125501
2,2020-11-10T17:00:00,3,Lombardia,6682,708,7390,128115,135505,4046,10955,122410,18571,213059.0,63427.0,276486,3324961,2039672.0,10103969
3,2020-11-10T17:00:00,4,Trentino-Alto Adige,717,65,782,10690,11472,84,618,13171,827,20337.0,5133.0,25470,572731,NaN,1074819
4,2020-11-10T17:00:00,5,Veneto,1596,204,1800,48450,50250,-2912,2763,31362,2643,26311.0,57944.0,84255,2479307,962389.0,4907704
5,2020-11-10T17:00:00,6,Friuli Venezia Giulia,336,47,383,7458,7841,9,482,7213,476,13482.0,2048.0,15530,589577,245027.0,1211357
6,2020-11-10T17:00:00,7,Liguria,1315,92,1407,11929,13336,615,1172,22828,1976,26677.0,11463.0,38140,496323,253455.0,1543127
7,2020-11-10T17:00:00,8,Emilia-Romagna,2020,215,2235,39545,41780,2150,2430,29230,4845,51310.0,24545.0,75855,1763487,931259.0,4467118
8,2020-11-10T17:00:00,9,Toscana,1582,242,1824,44490,46314,1834,2223,19451,1648,54431.0,12982.0,67413,1241973,813816.0,3722729
9,2020-11-10T17:00:00,10,Umbria,363,66,429,9711,10140,144,485,5392,206,4484.0,11254.0,15738,336098,192401.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-10T17:00:00,1,Piemonte,4715,325,5040,53795,58835,2579,3659,40737,4742,44444.0,59870.0,104314,1178337,745706.0,4341375,367
1,2020-11-10T17:00:00,2,Valle d'Aosta,161,15,176,2123,2299,33,127,1946,218,3936.0,527.0,4463,46630,27907.0,125501,20
2,2020-11-10T17:00:00,3,Lombardia,6682,708,7390,128115,135505,4046,10955,122410,18571,213059.0,63427.0,276486,3324961,2039672.0,10103969,983
3,2020-11-10T17:00:00,4,Trentino-Alto Adige,717,65,782,10690,11472,84,618,13171,827,20337.0,5133.0,25470,572731,NaN,1074819,106
4,2020-11-10T17:00:00,5,Veneto,1596,204,1800,48450,50250,-2912,2763,31362,2643,26311.0,57944.0,84255,2479307,962389.0,4907704,825
5,2020-11-10T17:00:00,6,Friuli Venezia Giulia,336,47,383,7458,7841,9,482,7213,476,13482.0,2048.0,15530,589577,245027.0,1211357,175
6,2020-11-10T17:00:00,7,Liguria,1315,92,1407,11929,13336,615,1172,22828,1976,26677.0,11463.0,38140,496323,253455.0,1543127,209
7,2020-11-10T17:00:00,8,Emilia-Romagna,2020,215,2235,39545,41780,2150,2430,29230,4845,51310.0,24545.0,75855,1763487,931259.0,4467118,516
8,2020-11-10T17:00:00,9,Toscana,1582,242,1824,44490,46314,1834,2223,19451,1648,54431.0,12982.0,67413,1241973,813816.0,3722729,415
9,2020-11-10T17:00:00,10,Umbria,363,66,429,9711,10140,144,485,5392,206,4484.0,11254.0,15738,336098,192401.0,880285,70


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-10T17:00:00,3,Lombardia,6682,708,7390,128115,135505,4046,10955,...,72.02,3740,6178,47194,25538.0,129,6780,23.21,0.10842,2.74
1,2020-11-10T17:00:00,1,Piemonte,4715,325,5040,53795,58835,2579,3659,...,88.56,2391,783,15812,10597.0,64,1016,23.14,0.08428,2.40
2,2020-11-10T17:00:00,5,Veneto,1596,204,1800,48450,50250,-2912,2763,...,24.73,-3053,540,16778,8453.0,48,5627,16.47,0.05630,1.72
3,2020-11-10T17:00:00,15,Campania,2061,193,2254,71974,74228,1908,2716,...,45.20,1794,-404,14290,11067.0,18,790,19.01,0.04694,1.60
4,2020-11-10T17:00:00,12,Lazio,2785,257,3042,52547,55589,2315,2608,...,34.40,2271,455,29315,24224.0,36,257,8.90,0.04446,1.20
5,2020-11-10T17:00:00,8,Emilia-Romagna,2020,215,2235,39545,41780,2150,2430,...,41.67,2058,405,22539,8956.0,21,252,10.78,0.05440,1.70
6,2020-11-10T17:00:00,9,Toscana,1582,242,1824,44490,46314,1834,2223,...,58.31,1786,-21,14248,7336.0,54,335,15.60,0.05971,1.81
7,2020-11-10T17:00:00,16,Puglia,1029,131,1160,17726,18886,1073,1245,...,35.79,1000,515,8825,6085.0,25,147,14.11,0.03106,0.69
8,2020-11-10T17:00:00,19,Sicilia,1348,195,1543,21289,22832,893,1201,...,36.25,840,178,8856,6013.0,32,276,13.56,0.02417,0.67
9,2020-11-10T17:00:00,7,Liguria,1315,92,1407,11929,13336,615,1172,...,44.02,687,634,7022,3513.0,39,518,16.69,0.07595,2.47


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,66,4,70,94.29
1,Piemonte,325,42,367,88.56
2,Valle d'Aosta,15,5,20,75.00
3,Lombardia,708,275,983,72.02
4,Trentino-Alto Adige,65,41,106,61.32
5,Marche,75,52,127,59.06
6,Toscana,242,173,415,58.31
7,Campania,193,234,427,45.20
8,Liguria,92,117,209,44.02
9,Emilia-Romagna,215,301,516,41.67


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])) * 100).round(3)
    return df

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ./data/csv/regioni/covid19-piemonte.csv
Aggiornato ./data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ./data/csv/regioni/covid19-lombardia.csv
Aggiornato ./data/csv/regioni/covid19-veneto.csv
Aggiornato ./data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ./data/csv/regioni/covid19-liguria.csv
Aggiornato ./data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ./data/csv/regioni/covid19-toscana.csv
Aggiornato ./data/csv/regioni/covid19-umbria.csv
Aggiornato ./data/csv/regioni/covid19-marche.csv
Aggiornato ./data/csv/regioni/covid19-lazio.csv
Aggiornato ./data/csv/regioni/covid19-abruzzo.csv
Aggiornato ./data/csv/regioni/covid19-molise.csv
Aggiornato ./data/csv/regioni/covid19-campania.csv
Aggiornato ./data/csv/regioni/covid19-puglia.csv
Aggiornato ./data/csv/regioni/covid19-basilicata.csv
Aggiornato ./data/csv/regioni/covid19-calabria.csv
Aggiornato ./data/csv/regioni/covid19-sicilia.csv
Aggiornato ./data/csv/regioni/covid19-sardegna.csv
